# Part 2

Can the connectome give us insights about how backwards walking works?

## Part 2A
In order to adress this question, we try to find groupings of motor neurons that are recruited synergystically during backwards walking, focusing on the hind legs. 

With those clusters identified, we can make predictions about which muscles should contract together, and which movemes compose the full stepping pattern.

## Part 2B 
We can also look at the connectivity of the neurons in the connectome to see if there are any patterns that emerge. For example, are the neurons that are recruited synergystically recruited by single 'hub' neurons, or are they recruited by multiple neurons? 

In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np

from vnc_networks import MANC, UID, CMatrix, Connections, params
from vnc_networks.specific_neurons import mdn_helper


In [ ]:
# ----- Directories
manc_version = "v1.0"
MDN_DIR = "MDN_project"
FIG_DIR = MANC(manc_version).get_fig_dir()
MDN_FIGS = os.path.join(FIG_DIR, MDN_DIR)
os.makedirs(MDN_FIGS, exist_ok=True)

In [ ]:
savefigs = False  # True to save figures

In [ ]:
CR = MANC(manc_version)
split_mdn_vnc = mdn_helper.get_vnc_split_MDNs_by_neuropil(
    not_connected=mdn_helper.get_mdn_bodyids()
)  # exclude connections from MDNs to MDNs

# Part 2A: MDN-induced motor neuron clustering

We need to compare the clustering of motor neurons in the graph induced by MDN input vs the one that exists naturally.

In [ ]:
# Method choices

# Method for clustering
clustering_method = "markov"  #'hierarchical'
# distance metric
distance_metric = "cosine_in"
# cutoff to deifne a cluster
cutoff = 0.5
# minimum number of neurons in a cluster
c_min = 4

# Analysis choices
side = "RHS"
leg = "hl"

In [ ]:
# Hind leg, right side leg motor neurons
motor_neurons = split_mdn_vnc.get_neuron_ids(
    {
        "class_1": "motor",
        "class_2": leg,
        "side": side,
    }
)
print(f"Found {len(motor_neurons)} motor neurons for the right hind leg")


## Part 2Aa: Control for right T3

In [ ]:
vnc_matrix = split_mdn_vnc.get_cmatrix(
    type_="unnorm"
)  # the weights are the signed synapse counts, no normalisation is done


In [ ]:
# Cut it down to the motor and premotor connections
premotor_neurons = vnc_matrix.list_upstream_neurons(motor_neurons)
nodes = list(set(motor_neurons).union(premotor_neurons))
vnc_matrix.restrict_nodes(nodes)

In [ ]:
# cluster the motor neurons using cosine similarity on the inputs
# Clustering
(
    clustered_cmatrix,  # clustered similarity matrix as cmatrix object
    uid_clusters,  # list of lists of uids in each cluster
    index_clusters,  # list of lists of indices in each cluster matching the clustered cmatrix
) = vnc_matrix.detect_clusters(
    distance=distance_metric,
    method=clustering_method,
    cutoff=cutoff,
    cluster_size_cutoff=c_min,
    cluster_data_type="uid",
    cluster_on_subset=motor_neurons,
)

In [ ]:
# Visualise the clusters

fig, ax = plt.subplots(figsize=(6, 6))
# Visualise the similarity matrix and its clusters
clustered_sim_mat = clustered_cmatrix.get_matrix().todense()
# create a matrix of zeros
mat = np.zeros((clustered_sim_mat.shape[0], clustered_sim_mat.shape[1]))
# draw the boundaries between clusters
for cluster in index_clusters:
    mat[cluster[0] : cluster[-1] + 1, cluster[0] : cluster[-1] + 1] = 1
_ = clustered_cmatrix.imshow(savefig=False, ax=ax, title="Clustered similarity matrix")
ax.imshow(mat, cmap="binary", alpha=0.2)

if savefigs:
    plt.savefig(
        os.path.join(MDN_FIGS,"clustered_similarity_matrix_T3_R_MNs_control.pdf"),
        dpi=params.DPI, bbox_inches="tight",
    )
plt.show()

We see XXX clusters in the control case. Observations?

In [ ]:
# Save the clusters
# create a df with a column for the cluster number, one for the neuron uid, one 
# for the neuron bodyid, and the rest for the defined neuron properties.
neurons_in_clusters = [uid for cluster in uid_clusters for uid in cluster]

# include 'target' and 'hemilineage' as these are useful for identifying the neurons
_ = split_mdn_vnc.get_node_attribute(uid=neurons_in_clusters, attribute="target")
_ = split_mdn_vnc.get_node_attribute(uid=neurons_in_clusters, attribute="hemilineage")

# retrieve the properties of the neurons in the clusters
info_df = split_mdn_vnc.list_neuron_properties(
    neurons=neurons_in_clusters,
    input_type="uid",
)
info_df["cluster"] = -1
for i, cluster in enumerate(uid_clusters):
    info_df.loc[info_df["uid"].isin(cluster), "cluster"] = i
info_df.sort_values(by=["cluster", "uid"], inplace=True)
print(info_df)
if savefigs:
    info_df.to_csv(
        os.path.join(MDN_FIGS, "motor_clusters_right_hind_leg_control.csv"), index=False
    )

Muscles synergies observed:
1. XXX
2. YYY

## Part 2Ab: MDN circuit for right T3

In [ ]:
# Select the MDN subdivisions that have synapses in the hind right leg
input_neurons = mdn_helper.get_subdivided_mdns(
    VNC=split_mdn_vnc,
    neuropil=leg,
    side=side,
)

In [ ]:
# Keep only the connections that create a path from source to target

subnetwork = split_mdn_vnc.subgraph_from_paths(  # copy operation
    source=input_neurons,
    target=motor_neurons,
    n_hops=2,  # within 2 hops, i.e. only 1 interneuron
    keep_edges="intermediate",  # keep the connections between the interneurons
    # as well, but not between source neurons or between target neurons
    # can also be 'direct' (only direct paths) or 'all' (all connections between
    # recruited nodes)
)

In [ ]:
# Get the connectivity matrix

subnetwork_matrix = subnetwork.get_cmatrix(type_="unnorm")
# Cut it down to the motor and premotor connections
premotor_neurons = subnetwork_matrix.list_upstream_neurons(motor_neurons)
nodes = list(set(motor_neurons).union(premotor_neurons))
subnetwork_matrix.restrict_nodes(nodes)


In [ ]:
# cluster the motor neurons using cosine similarity on the inputs

(
    sub_clustered_cmatrix,  # clustered similarity matrix as cmatrix object
    sub_uid_clusters,  # list of lists of uids in each cluster
    sub_index_clusters,  # list of lists of indices in each cluster matching the clustered cmatrix
) = subnetwork_matrix.detect_clusters(
    distance=distance_metric,
    method=clustering_method,
    cutoff=cutoff,
    cluster_size_cutoff=c_min,
    cluster_data_type="uid",
    cluster_on_subset=motor_neurons,
)

In [ ]:
# Visualise the similarity matrix and its clusters

fig, ax = plt.subplots(figsize=(6, 6))
sub_clustered_sim_mat = sub_clustered_cmatrix.get_matrix().todense()
# create a matrix of zeros
mat = np.zeros((sub_clustered_sim_mat.shape[0], sub_clustered_sim_mat.shape[1]))
# draw the boundaries between clusters
for cluster in sub_index_clusters:
    mat[cluster[0] : cluster[-1] + 1, cluster[0] : cluster[-1] + 1] = 1
_ = sub_clustered_cmatrix.imshow(
    savefig=False, ax=ax, title="Clustered similarity matrix"
)
ax.imshow(mat, cmap="binary", alpha=0.2)

if savefigs:
    fig.savefig(
        os.path.join(
            MDN_FIGS, "clustered_similarity_matrix_T3_R_MNs_MDN_input.pdf", dpi=300
        ),
        dpi=params.DPI,
        bbox_inches="tight",
    )

plt.show()


Observations?

In [ ]:
# Save the clusters
# create a df with a column for the cluster number, one for the neuron uid, one
# for the neuron bodyid, and the rest for the defined neuron properties.
neurons_in_subclusters = [uid for cluster in sub_uid_clusters for uid in cluster]

sub_info_df = split_mdn_vnc.list_neuron_properties(
    neurons=neurons_in_subclusters,
    input_type="uid",
)
sub_info_df["cluster"] = -1
for i, cluster in enumerate(sub_uid_clusters):
    sub_info_df.loc[sub_info_df["uid"].isin(cluster), "cluster"] = i
sub_info_df.sort_values(by=["cluster", "uid"], inplace=True)
print(sub_info_df)
if savefigs:
    sub_info_df.to_csv(
        os.path.join(MDN_FIGS, "motor_clusters_right_hind_leg_MDN_input.csv"),
        index=False,
    )


Observed clusters:
1. XXX
2. YYY

# Part 2B: premotor connectivity analysis

## Part 2Ba: contribution of individual premotor neurons

We want to see which premotor neurons contribute to the recruitment of the motor neurons in the clusters we identified in Part 2A.

In order to do so, we define a metric `m1` that quantifies the unique contribution of a premotor neuron to the recruitement of a cluster as following:
- sum the absolute values of the weights (i.e. synapse count) of the edges between the premotor neuron and the motor neurons in the cluster
- substract the sum of the absolute values of the weights of the edges between the premotor neuron and the motor neurons not in the cluster, scaled by a discount factor `alpha` (e.g. nb. of motor neurons in the cluster divided by the number of remaining motor neurons, such that if all connections are equal the score is 0)

In [ ]:
def m1(
        cmat: CMatrix, # connectivity matrix
        source: UID, # premotor neuron
        targets: list[UID], # neurons in cluster
        motor_neurons: list[UID], # all motor neurons
        ):
        """
        M1 metric as defined above.
        """
        m1 = 0

        # Positive contributions to the targets
        direct_mat = cmat.get_matrix(
                row_ids=[source],
                column_ids=targets,
                input_type="uid",
                )
        direct_mat = abs(direct_mat)
        m1 += direct_mat.sum()

        # Negative contributions to the targets
        other_targets = [mn for mn in motor_neurons if mn not in targets]
        other_connections = cmat.get_matrix(
                row_ids = [source],
                column_ids = other_targets,
                input_type = "uid",
        )
        other_connections = abs(other_connections)

        # alpha: #targets/#non targets
        alpha = len(targets) / len(other_targets)
        m1 -= alpha * other_connections.sum()

        return m1
        

In [ ]:
# Load the motor neuron clusters

mdn_based_clustering_df = sub_info_df.copy() # If no saving to file
# mdn_based_clustering_df = pd.read_csv( # If saving to file 
#     os.path.join(MDN_FIGS, "motor_clusters_right_hind_leg_control.csv")
# )
def get_uids_from_cluster(cluster_number, df):
    return df[df["cluster"] == cluster_number]["uid"].values

In [ ]:
# Add nt_type to read node properties
_ = split_mdn_vnc.get_node_attribute(uid=neurons_in_clusters, attribute="nt_type")


In [ ]:
# visualise contribution for each cluster
clusters = np.unique(mdn_based_clustering_df['cluster'].values)
nb_clusters = len(clusters)

fig, axs = plt.subplots(1, nb_clusters, figsize=(30,6))

for i, c in enumerate(clusters):
    c_i = get_uids_from_cluster(c, mdn_based_clustering_df)
    c1_m1 = [
        m1(cmat=subnetwork_matrix, source=pmn, targets=c_i, motor_neurons=motor_neurons)
        for pmn in premotor_neurons
        ]
    x = np.linspace(0, len(c1_m1), len(c1_m1))
    nts = split_mdn_vnc.get_node_attribute(uid=premotor_neurons,attribute='nt_type')
    colors = [params.NT_TYPES[nt]["color"] for nt in nts]
    axs[i].scatter(x,c1_m1, color = colors, marker = 'o', linestyle='None', label=f'cluster {c}')
    # get error shading
    std_i = 3*np.std(c1_m1) # 99th percentile
    mean_i = np.mean(c1_m1)
    axs[i].axhline(y=mean_i, linewidth=4, color='k')
    x = np.linspace(0, len(c1_m1), len(c1_m1))
    axs[i].fill_between(x,mean_i-std_i, mean_i+std_i, alpha=0.1, color='grey')
    axs[i].legend()

    # print the neurons that contribute significantly
    significant_neurons = [
        node for i, node in enumerate(premotor_neurons)
        if c1_m1[i] > mean_i + std_i
        ]
    info = split_mdn_vnc.list_neuron_properties(
        neurons=significant_neurons,
        input_type="uid",
    )
    info["m1 score"] = info["uid"].apply(
        lambda x: m1(
            cmat=subnetwork_matrix,
            source=x,
            targets=c_i,
            motor_neurons=motor_neurons,
        )
    )
    info.sort_values(by="m1 score", ascending = False, inplace=True)
    print(f"Cluster {c} significant contributors:")
    print(info)
    print('\n')
    

plt.show()


With the exception of one cluster that is significantly driven by 2 excitatory neurons, the other 5 clusters have as sole out-of-distribution contributors a single gaba-ergic neuron. There are clear hubs driving those movements, and we can hypothesise that what matters is a cycle desinhibition.